## 1. Import all libraries


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge folium=0.5.0 --yes
import requests

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Libraries have been imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
address = 'Whitefield, Bangalore, India'
geolocator = Nominatim(user_agent='Bang_explorer')
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude

whitefield_lat = lat
whitefield_lng = lng
print("The coordinates of Whitefield neighborhood in Bangalore are {},{}.".format(lat,lng))

The coordinates of Whitefield neighborhood in Bangalore are 12.9696365,77.7497448.


In [3]:
CLIENT_ID='Your Client ID'
CLIENT_SECRET='Your Client Secret'
VERSION=20200701
radius = 8000
LIMIT=100

In [4]:
search_query = 'Petrol Pump'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,lat,lng,VERSION,search_query,radius,LIMIT)

results = requests.get(url).json()

In [5]:
gas_station = results['response']['venues']
df = json_normalize(gas_station)
df.columns = [col.split('.')[-1] for col in df.columns]
df2 = df[['name','categories','lat','lng','id','distance']]
df2.head()

,name,categories,lat,lng,id,distance
0,Shell Petrol Pump,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",12.956405,77.731320,4d4521271b62b1f71f85fae2,2482
1,Shell Petrol Pump,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",13.017656,77.762003,516cd563e4b03dde554c50d0,5508
2,Shell Petrol Pump,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",12.931880,77.742415,5cf4f68586f4cc00369effab,4277
3,BP Petrol pump,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",12.994544,77.716590,4d470da9e198721e1a85d58b,4541
4,Shell Petrol Pump,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",12.895566,77.723076,5af817c55bf1b300302248d2,8738


In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
df2['categories'] = df2.apply(get_category_type,axis=1)
df2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,name,categories,lat,lng,id,distance
0,Shell Petrol Pump,Gas Station,12.956405,77.731320,4d4521271b62b1f71f85fae2,2482
1,Shell Petrol Pump,Gas Station,13.017656,77.762003,516cd563e4b03dde554c50d0,5508
2,Shell Petrol Pump,Gas Station,12.931880,77.742415,5cf4f68586f4cc00369effab,4277
3,BP Petrol pump,Gas Station,12.994544,77.716590,4d470da9e198721e1a85d58b,4541
4,Shell Petrol Pump,Gas Station,12.895566,77.723076,5af817c55bf1b300302248d2,8738


In [8]:
df2.columns = ['Name','Category','Latitude','Longitude','Id','Distance']
df2.head()

,Name,Category,Latitude,Longitude,Id,Distance
0,Shell Petrol Pump,Gas Station,12.956405,77.731320,4d4521271b62b1f71f85fae2,2482
1,Shell Petrol Pump,Gas Station,13.017656,77.762003,516cd563e4b03dde554c50d0,5508
2,Shell Petrol Pump,Gas Station,12.931880,77.742415,5cf4f68586f4cc00369effab,4277
3,BP Petrol pump,Gas Station,12.994544,77.716590,4d470da9e198721e1a85d58b,4541
4,Shell Petrol Pump,Gas Station,12.895566,77.723076,5af817c55bf1b300302248d2,8738


In [9]:
import folium
bang_map = folium.Map(location=[whitefield_lat,whitefield_lng],zoom_start=12)

folium.features.CircleMarker([whitefield_lat,whitefield_lng], 
                                radius=10,
                                color='red',                            
                                popup = 'You are here!', 
                                fill = True, 
                                fill_color='red', 
                                fill_opacity = 0.7
                                ).add_to(bang_map)
bang_map

In [10]:
from folium import plugins

incidents = folium.plugins.MarkerCluster().add_to(bang_map)

for lat,lng,label in zip(df2['Latitude'],df2['Longitude'],df2['Name']):
    folium.Marker(
        location = [lat,lng],
        icon = None,
        popup = label,
        ).add_to(incidents)

bang_map

## Now let's try to create dataframe for 2 categories

In [11]:
search_query=['Petrol Pump', 'Hospital', 'Pharmacy', 'ATM Machine', 'Shopping Mall','Post Office', 'Banks', 'Gym']
#search_query=['Petrol Pump','Hospital']

In [12]:
def get_essentials_venues(search_list):
    essentials_list = []

    for squery in search_list:
        #print(squery)
        urlx = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,whitefield_lat,whitefield_lng,VERSION,squery,radius,LIMIT)
        results = requests.get(urlx).json()['response']['venues']
        essentials_list.append([(v['name'],
                              v['location']['lat'],
                              v['location']['lng'],
                              v['categories'], squery) for v in results])

    essential_venues = pd.DataFrame([item for essential_list in essentials_list for item in essential_list])
    essential_venues.columns = ['Name','Latitude','Longitude','categories','Search Query']
    essential_venues['categories'] = essential_venues.apply(get_category_type,axis=1)
    essential_venues.columns = ['Name','Latitude','Longitude','Category', 'Search Query']
    return (essential_venues)

In [13]:
whitefield_essential_venues = get_essentials_venues(search_query)

In [14]:
whitefield_essential_venues.shape

(293, 5)

In [15]:
whitefield_essential_venues.head(25)

,Name,Latitude,Longitude,Category,Search Query
0,Shell Petrol Pump,12.956405,77.731320,Gas Station,Petrol Pump
1,Shell Petrol Pump,13.017656,77.762003,Gas Station,Petrol Pump
2,Shell Petrol Pump,12.931880,77.742415,Gas Station,Petrol Pump
3,BP Petrol pump,12.994544,77.716590,Gas Station,Petrol Pump
4,Shell Petrol Pump,12.895566,77.723076,Gas Station,Petrol Pump
5,Essar Petrol Pump,13.014133,77.709690,Gas Station,Petrol Pump
6,HP Petrol Pump,12.996859,77.691681,Gas Station,Petrol Pump
7,HP Petrol Pump Kaggadasapura,12.978297,77.679602,Gas Station,Petrol Pump
8,Shell Petrol Pump,12.971227,77.682005,Gas Station,Petrol Pump
9,Hp Petrol Bunk,12.956154,77.745120,Gas Station,Petrol Pump


## Now let's get the essential venues for all the neighborhoods in Bangalore

In [16]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_06dbcd6206b1422394d5db06212cdb35 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='v3L9gN_0iTGojxmknnVkKi0tltuLhpIcgqgvo30_Bqf_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_06dbcd6206b1422394d5db06212cdb35.get_object(Bucket='courseracapstoneproject2-donotdelete-pr-wok2dygo0tijlg',Key='bangalore_data.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


,Post office,Office type,Pincode
0,A F Station Yelahanka,S.O,560063
1,Adarangi,B.O,561101
2,Adugodi,S.O,560030
3,Agram,S.O,560007
4,Air Force Hospital,S.O,560007


In [17]:
location_data = []
for po,pincode in zip(df_data_0['Post office'],df_data_0['Pincode']):
    address = pincode
    geolocator = Nominatim(user_agent='Bangalore_Explorer')
    location = geolocator.geocode(address)
    location_data.append([po, location.latitude,location.longitude])

In [18]:
df_bangalore= pd.DataFrame(location_data)
df_bangalore.columns = ['Neighborhood','Latitude','Longitude']

In [19]:
df_bangalore.head(7)

,Neighborhood,Latitude,Longitude
0,A F Station Yelahanka,13.060937,77.614362
1,Adarangi,13.131968,77.148830
2,Adugodi,12.941853,77.602391
3,Agram,12.965107,77.629029
4,Air Force Hospital,12.965107,77.629029
5,Ajjanahalli,12.900263,77.278009
6,Alurduddanahalli,13.257421,77.619129


In [20]:
df_test = df_bangalore.loc[:5,:]

In [21]:
df_test.head()

,Neighborhood,Latitude,Longitude
0,A F Station Yelahanka,13.060937,77.614362
1,Adarangi,13.131968,77.148830
2,Adugodi,12.941853,77.602391
3,Agram,12.965107,77.629029
4,Air Force Hospital,12.965107,77.629029


In [22]:
df_test['Latitude']

0    13.060937
1    13.131968
2    12.941853
3    12.965107
4    12.965107
5    12.900263
Name: Latitude, dtype: float64

In [23]:
def get_essentials_venues2(search_list, Neighborhoods, Latitudes, Longitudes):
    essentials_list2 = []
    for nh,lat,lng in zip(Neighborhoods,Latitudes,Longitudes):
        for squery in search_list:
            #print(squery)
            urlx = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,lat,lng,VERSION,squery,radius,LIMIT)
            results = requests.get(urlx).json()['response']['venues']
            essentials_list2.append([(nh, lat, lng,
                                  v['name'],
                                  v['location']['lat'],
                                  v['location']['lng'],
                                  v['categories'], squery) for v in results])

            essential_venues2 = pd.DataFrame([item for essential_list in essentials_list2 for item in essential_list])
    return (essential_venues2)

In [24]:
bangalore_essential_venues = get_essentials_venues2(search_query,df_test['Neighborhood'],df_test['Latitude'],df_test['Longitude'])

In [25]:
bangalore_essential_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Name','Latitude','Longitude','categories','Search Query']

In [26]:
bangalore_essential_venues['categories'] = bangalore_essential_venues.apply(get_category_type,axis=1)
bangalore_essential_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Name','Latitude','Longitude','Category', 'Search Query']


In [27]:
bangalore_essential_venues.shape

(1429, 8)

## So we have generated a dataframe with essential locations for 6 neighborhoods in Bangalore

In [29]:
bangalore_essential_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name,Latitude,Longitude,Category,Search Query
1424,Air Force Hospital,12.965107,77.629029,Lemon Tree Hotel Gym,12.987274,77.616743,Gym / Fitness Center,Gym
1425,Air Force Hospital,12.965107,77.629029,Gym Plus,12.994139,77.607751,Recreation Center,Gym
1426,Air Force Hospital,12.965107,77.629029,gym park,12.927957,77.627251,Athletics & Sports,Gym
1427,Air Force Hospital,12.965107,77.629029,Catholic Club Gym,12.969434,77.605986,Gym,Gym
1428,Ajjanahalli,12.900263,77.278009,Apollo Pharmacy,12.958058,77.219440,Pharmacy,Pharmacy


In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors
search_query=['Petrol Pump', 'Hospital', 'Pharmacy', 'ATM Machine', 'Shopping Mall','Post Office', 'Banks', 'Gym']
x = np.arange(len(search_query))
#print('x = ',x)
ys = [i+x+(i*x)**2 for i in range(len(search_query))]
#print(ys)

colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
marker_colors = []

bang_map2 = folium.Map(location = [whitefield_lat,whitefield_lng],zoom_start=11)

for lati,longi,lbl,nb,sq in zip(bangalore_essential_venues['Latitude'],bangalore_essential_venues['Longitude'],bangalore_essential_venues['Name'],bangalore_essential_venues['Neighborhood'],bangalore_essential_venues['Search Query']):
    z = search_query.index(sq)
    label = folium.Popup(str(lbl) + str(',') + str(nb), parse_html = True)
    folium.features.CircleMarker([lati,longi],
                                 radius = 5,
                                popup = label,
                                color = rainbow[z],
                                fill = True,
                                fill_color = rainbow[z],
                                fill_opacity = 0.7,
                                parse_html = False
                                    ).add_to(bang_map2)

bang_map2
